In [ ]:
import geopandas as gpd

import pandas as pd

In [ ]:
# filter by origin code 1(native) or 2 (reintroduced to native range)
# filter by presence code 1 (present/extant)

In [ ]:
zip = "/media/muskrat/T7 Shield/eco_data/v3/native/IUCN/BONEFISH_TARPONS.zip"

df = gpd.read_file(zip)

In [ ]:
# print length of dataframe
len(df)

In [ ]:
df.columns

In [ ]:
# some are Shape_Leng and Shape_Area instead of SHAPE_Leng and SHAPE_Area
# if df contains columns SHAPE_Leng and SHAPE_Area, rename them to Shape_Leng and Shape_Area
if "SHAPE_Leng" in df.columns and "SHAPE_Area" in df.columns:
    df = df.rename(columns={"SHAPE_Leng": "Shape_Leng", "SHAPE_Area": "Shape_Area"})

df = df.drop(
    columns=[
        "id_no",
        "seasonal",
        "compiler",
        "yrcompiled",
        "subspecies",
        "subpop",
        "Shape_Leng",
        "Shape_Area",
        "source",
        "island",
        "tax_comm",
        "dist_comm",
        "generalisd",
        "legend",
    ]
)

df.columns

In [ ]:
df = df.drop(
    columns=[
        "citation",
        "category",
        "marine",
        "terrestial",
        "freshwater",
        "kingdom",
        "phylum",
        "class",
        "order_",
        "family",
        "genus",
    ]
)

df.columns

In [ ]:
# print unique values in presence column
df["presence"].unique()

In [ ]:
# remove rows of df where column 'presence' is not 1 or 2
df = df[df["presence"].isin([1, 2])]

df["presence"].unique()

In [ ]:
# remove rows of df where column 'origin' is not 1
df = df[df["origin"].isin([1])]

In [ ]:
len(df)

In [ ]:
df

In [ ]:
# create new df sci_df with columns 'sci_name', 'presence', and 'origin'
sci_df = df[["sci_name", "presence", "origin"]]

sci_df

In [ ]:
df = df.to_crs("EPSG:4326")

df.crs

In [ ]:
unique_species = df["sci_name"].unique().tolist()

len(unique_species)

In [ ]:
# remove items in unique_species that contain more than 2 words
unique_species = [x for x in unique_species if len(x.split()) < 3]

len(unique_species), unique_species

In [ ]:
# sort unique_species alphabetically
unique_species.sort()

unique_species

In [ ]:
for i in range(0, len(unique_species), 1):
    species = unique_species[i]
    species_df = df[df["sci_name"] == species]
    print(species_df)

    # overlay = gpd.overlay(species_df, eco_map, how="intersection")

    # unique_ids = list(overlay["unique_id"].unique())

    # eco_map_unique = eco_map[eco_map["unique_id"].isin(unique_ids)]

    # # remove rows with <NA> in unique_id from eco_map_unique

    # eco_map_unique = eco_map_unique[eco_map_unique["unique_id"] != "<NA>"]

    # overlay = overlay[overlay["unique_id"] != "<NA>"]

    # overlay["area"] = overlay.geometry.area

    # overlay_areas = overlay[["unique_id", "area"]].groupby("unique_id").sum()

    # eco_map_unique["area"] = eco_map_unique.geometry.area

    # eco_map_unique_areas = (
    #     eco_map_unique[["unique_id", "area"]].groupby("unique_id").sum()
    # )

    # combined_areas = pd.concat([overlay_areas, eco_map_unique_areas], axis=1)
    # combined_areas.columns = ["overlay_area", "eco_map_unique_area"]

    # combined_areas["ratio"] = (
    #     combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"]
    # )

    # # keep ecoregion if only present in 1 area
    # if len(combined_areas) == 1:
    #     native = unique_ids
    # else:
    #     # if ratio > 0.2 then add unique_id to list of ids
    #     native = combined_areas[combined_areas["ratio"] > 0.2].index.tolist()

    #     if len(native) == 0:

    #         native = combined_areas[
    #             combined_areas["eco_map_unique_area"] <= 2
    #         ].index.tolist()

    #         if len(native) == 0:
    #             native = combined_areas[combined_areas["ratio"] > 0.1].index.tolist()

    #             if len(native) == 0:
    #                 native = ["check"]

    # scientific_name = species_df["sci_name"].values[0]

    # species_final = pd.DataFrame(
    #     {
    #         "scientific_name": scientific_name,
    #         "database": "iucn",
    #         "unique_id": native,
    #     },
    # )

    # # groupby scientific_name
    # species_final = (
    #     species_final.groupby(["scientific_name", "database"])["unique_id"]
    #     .apply(list)
    #     .reset_index()
    # )

In [ ]:
# put first row of df into a new dataframe
df2 = df[df["sci_name"] == "Limnonectes hascheanus"]
df2

In [ ]:
df2.plot()

In [ ]:
ecomap_loc = "/media/muskrat/T7 Shield/eco_data/ecomap_final/eco_map.geojson"

eco_map = gpd.read_file(ecomap_loc)

In [ ]:
# plot ecomap and df on same map
base = eco_map.plot(color="white", edgecolor="black")
xmin, ymin, xmax, ymax = (90, 0, 110, 30)

ax = df2.plot(ax=base, color="red", alpha=0.4)

# set the x and y limits of the plot to the specified bounding box coordinates
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
# ax.invert_xaxis()

In [ ]:
intersects = gpd.sjoin(df2, eco_map)

unique_ids = list(intersects["unique_id"].unique())

unique_ids

In [ ]:
overlay = gpd.overlay(df2, eco_map, how="intersection")

overlay_ids = list(overlay["unique_id"].unique())

overlay_ids

In [ ]:
overlay = gpd.overlay(df2, eco_map, how="intersection")

unique_ids = list(overlay["unique_id"].unique())

eco_map_unique = eco_map[eco_map["unique_id"].isin(unique_ids)]

# remove rows with <NA> in unique_id from eco_map_unique

eco_map_unique = eco_map_unique[eco_map_unique["unique_id"] != "<NA>"]

overlay = overlay[overlay["unique_id"] != "<NA>"]

# overlay.plot(alpha=0.5, edgecolor="k", cmap="tab10")

overlay["area"] = overlay.geometry.area

overlay_areas = overlay[["unique_id", "area"]].groupby("unique_id").sum()

eco_map_unique["area"] = eco_map_unique.geometry.area

eco_map_unique_areas = eco_map_unique[["unique_id", "area"]].groupby("unique_id").sum()

combined_areas = pd.concat([overlay_areas, eco_map_unique_areas], axis=1)
combined_areas.columns = ["overlay_area", "eco_map_unique_area"]

combined_areas["ratio"] = (
    combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"]
)

combined_areas

In [ ]:
# keep ecoregion if only present in 1 area
if len(combined_areas) == 1:
    native = unique_ids
else:
    # if ratio > 0.2 then add unique_id to list of ids
    native = combined_areas[combined_areas["ratio"] > 0.2].index.tolist()

    if len(native) == 0:

        native = combined_areas[
            combined_areas["eco_map_unique_area"] <= 2
        ].index.tolist()

        if len(native) == 0:
            native = combined_areas[combined_areas["ratio"] > 0.1].index.tolist()

            if len(native) == 0:
                native = ["check"]

native

In [ ]:
scientific_name = df2["sci_name"].values[0]


final = pd.DataFrame(
    {
        "scientific_name": scientific_name,
        "database": "iucn",
        "unique_id": native,
    },
)

# groupby scientific_name
final = (
    final.groupby(["scientific_name", "database"])["unique_id"]
    .apply(list)
    .reset_index()
)


final

In [ ]:
sci_name = df2["sci_name"].values[0]
native = pd.DataFrame({"sci_name": sci_name, "unique_id": native})


native

In [ ]:
# groupby scientific_name
native = native.groupby(["sci_name"])["unique_id"].apply(list).reset_index()

native

In [ ]:
# merge native with df2
df2 = pd.merge(df2, native, on="sci_name")

df2

In [ ]:
df2 = df2.drop(columns=["presence", "origin", "geometry"])

df2

In [ ]:
# rename sci_name to scientific_name and order_ to order

df2 = df2.rename(columns={"sci_name": "scientific_name", "order_": "order"})

df2

In [ ]:
zip = input("Enter the zip file path: ")

df = gpd.read_file(zip)

ecomap_loc = input("Enter the ecomap file path: ")

eco_map = gpd.read_file(ecomap_loc)

# some are Shape_Leng and Shape_Area instead of SHAPE_Leng and SHAPE_Area
# if df contains columns SHAPE_Leng and SHAPE_Area, rename them to Shape_Leng and Shape_Area
if "SHAPE_Leng" in df.columns:
    df = df.rename(columns={"SHAPE_Leng": "Shape_Leng"})

if "SHAPE_Area" in df.columns:
    df = df.rename(columns={"SHAPE_Area": "Shape_Area"})


df = df.drop(
    columns=[
        "id_no",
        "seasonal",
        "compiler",
        "yrcompiled",
        "subspecies",
        "subpop",
        "Shape_Leng",
        "Shape_Area",
        "source",
        "island",
        "tax_comm",
        "dist_comm",
        "generalisd",
        "legend",
    ]
)

# remove rows of df where column 'presence' is not 1 or 2
df = df[df["presence"].isin([1, 2])]

# remove rows of df where column 'origin' is not 1
df = df[df["origin"].isin([1])]

df = df.to_crs("EPSG:4326")

unique_species = df["sci_name"].unique().tolist()

# remove items in unique_species that contain more than 2 words
unique_species = [x for x in unique_species if len(x.split()) < 3]

final = pd.DataFrame()
for species in unique_species:
    species_df = df[df["sci_name"] == species]

    intersects = gpd.sjoin(species_df, eco_map)

    unique_ids = list(intersects["unique_id"].unique())

    eco_map_unique = eco_map[eco_map["unique_id"].isin(unique_ids)]

    # remove rows with <NA> in unique_id from eco_map_unique

    eco_map_unique = eco_map_unique[eco_map_unique["unique_id"] != "<NA>"]

    overlay = gpd.overlay(species_df, eco_map, how="intersection")

    overlay = overlay[overlay["unique_id"] != "<NA>"]

    # overlay.plot(alpha=0.5, edgecolor="k", cmap="tab10")

    overlay["area"] = overlay.geometry.area

    overlay_areas = overlay[["unique_id", "area"]].groupby("unique_id").sum()

    eco_map_unique["area"] = eco_map_unique.geometry.area

    eco_map_unique_areas = (
        eco_map_unique[["unique_id", "area"]].groupby("unique_id").sum()
    )

    combined_areas = pd.concat([overlay_areas, eco_map_unique_areas], axis=1)
    combined_areas.columns = ["overlay_area", "eco_map_unique_area"]

    combined_areas["percentage"] = (
        combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"]
    )

    # combined_areas

    # keep ecoregion if only present in 1 area
    if len(combined_areas) == 1:
        native = unique_ids
    else:
        # if overlay_area / eco_map_unique_area > 0.2 then add unique_id to list of ids
        native = combined_areas[
            combined_areas["overlay_area"] / combined_areas["eco_map_unique_area"] > 0.2
        ].index.tolist()

        if len(native) == 0:

            native = combined_areas[
                combined_areas["eco_map_unique_area"] <= 2
            ].index.tolist()

            if len(native) == 0:
                native = combined_areas[
                    combined_areas["overlay_area"]
                    / combined_areas["eco_map_unique_area"]
                    > 0.1
                ].index.tolist()

                if len(native) == 0:
                    native = ["check"]

    scientific_name = species_df["sci_name"].values[0]

    native_df = pd.DataFrame({"sci_name": scientific_name, "unique_id": native})

    native_df = native_df.groupby(["sci_name"])["unique_id"].apply(list).reset_index()

    species_final = pd.merge(species_df, native_df, on="sci_name")

    species_final = species_final.drop(columns=["presence", "origin", "geometry"])

    species_final = species_final.rename(
        columns={"sci_name": "scientific_name", "order_": "order"}
    )

    final = pd.concat([final, species_final], axis=0)

In [ ]:
final

In [ ]:
final_path = input("Enter the file path: ")

final.to_parquet(final_path)